# Vessels making voyages
The `voyages` table contains top level information about a voyage from one port to another, including when and where the voyage started and ended, and which vessel was involved in the voyage. You can use this information to identify which vessels made a voyage from one port to another in some time range:

In [ ]:
from descarteslabs.vektorius import vector
from ipyleaflet import Map, GeoJSON
import warnings
warnings.filterwarnings('ignore')

ports = vector.table("ports")
voyages = vector.table("voyages")

# find geometry of start/end ports  that we care about
origin_port_name = "PORT DE SALVADOR"
dest_port_name = "PORTO DE SUAPE"
ports = ports.filter(ports.port_name.isin([origin_port_name, dest_port_name]))
port_geoms = ports[ports.port_name, ports.port_geom].execute()
start_port_geom = port_geoms[port_geoms.port_name == origin_port_name].port_geom.iloc[0]
end_port_geom = port_geoms[port_geoms.port_name == dest_port_name].port_geom.iloc[0]

# because we store Geographies as geodesics, we need to force these literal shapes
# to be treated as such
start_port_geom.crs = "EPSG:4326"
end_port_geom.crs = "EPSG:4326"

# find voyages that start or end within timeframe
start_date = "2018-01-01"
end_date = "2018-12-31"
dt_filter = (voyages.departure.between(start_date, end_date) |
             voyages.arrival.between(start_date, end_date))
    
# voyage should be between the two ports you care about
spatial_filter = (voyages.origin.intersects(start_port_geom) &
                  voyages.destination.intersects(end_port_geom))
    
voyages = voyages.filter(dt_filter & spatial_filter)
df = voyages[voyages.mmsi, voyages.linestring].execute()
df

Now we'll put this on a map to see what the voyages look like:

In [ ]:
# create a map with ipyleaflet
m = Map(center=(15.453680224345835, -23.466796875000004), zoom=3)

# __geo_interface__ allows us to get the GeoJSON FeatureCollection
# representation of the GeoSeries
geo_json = GeoJSON(data=df.linestring.__geo_interface__)
m.add_layer(geo_json)
m

You can also estimate how much cargo in kg was transported from one port to another in each month. You can use functions to alter column data to achieve this analysis before fetching any of the data locally:

In [ ]:
vessels = vector.table("vessels")

joined = voyages.inner_join(vessels, vessels.mmsi == voyages.mmsi)

capacity = joined[
    # get vessel capacity in kg
    (vessels.capacity.dwt * 1000).name("vessel_capacity_in_kg"),
    # get the departure month for nicer grouping
    voyages.departure.month().name("departure_month")
]
grouped = capacity.group_by(capacity.departure_month)
grouped = grouped.aggregate(capacity.vessel_capacity_in_kg
                            .sum()
                            .name("est_cargo_weight_in_kg"))
grouped = grouped.sort_by(grouped.departure_month)
df = grouped.execute()
df